# About: マシン確保(Google Compute Engine)

Google Compute Engine上でマシンの確保をおこなうためのNotebook。

## Operation Note

*ここに経緯を記述*

# 動作環境の確認

このNotebookは、 [Google Python Client Library](https://github.com/google/google-api-python-client) を使ってマシンの確保を行います。

このNotebook環境にGoogle Python Client Libraryがインストールされている必要があります。インストールされていない場合は、以下のセル実行に失敗し、 `ImportError` となります。

In [1]:
from googleapiclient import discovery

もし、インストールされていない場合は、[Using the Python Client Library](https://cloud.google.com/compute/docs/tutorials/python-guide)を参考にインストールしてください。

In [2]:
#!pip2 install --upgrade google-api-python-client
#from googleapiclient import discovery

# 設定の定義

どのようなマシンを確保するか？を定義していく。

## Credentialの指定

Google Compute EngineにアクセスするためのCredentialを指示する。

- JSONフォーマットのService Account情報
- プロジェクトID

を用意しておく。

In [3]:
creds = '~/.keys/xxxxxxxx.json'
target_project_id = 'my_project_id'

computeサービスを取得しておく。

In [4]:
import os
from oauth2client.client import GoogleCredentials

credentials = GoogleCredentials.from_stream(os.path.expanduser(creds))
compute = discovery.build('compute', 'v1', credentials=credentials)
compute

## ゾーンの設定

どのZoneにインスタンスを確保するかを定義しておく。

In [5]:
target_zone = 'us-central1-f'

## マシン名の決定

マシン名を決める。まず、現在のインスタンス名の一覧を確認する。

In [6]:
instances = compute.instances().list(zone=target_zone, project=target_project_id).execute()
map(lambda i: i['name'], instances['items'] if 'items' in instances else [])

[]

すでにあるインスタンスとは重複しないような名前を設定する。

In [7]:
machine_name = 'test-gce'

## マシンタイプの指定

まず、このZoneで利用可能なMachine Typeを取得する。

In [8]:
machineTypes = compute.machineTypes().list(project=target_project_id, zone=target_zone).execute()['items']
map(lambda t: (t['name'], t['description']), machineTypes)

[(u'f1-micro', u'1 vCPU (shared physical core) and 0.6 GB RAM'),
 (u'g1-small', u'1 vCPU (shared physical core) and 1.7 GB RAM'),
 (u'n1-highcpu-16', u'16 vCPUs, 14.4 GB RAM'),
 (u'n1-highcpu-2', u'2 vCPUs, 1.8 GB RAM'),
 (u'n1-highcpu-32', u'32 vCPUs, 28.8 GB RAM'),
 (u'n1-highcpu-4', u'4 vCPUs, 3.6 GB RAM'),
 (u'n1-highcpu-8', u'8 vCPUs, 7.2 GB RAM'),
 (u'n1-highmem-16', u'16 vCPUs, 104 GB RAM'),
 (u'n1-highmem-2', u'2 vCPUs, 13 GB RAM'),
 (u'n1-highmem-32', u'32 vCPUs, 208 GB RAM'),
 (u'n1-highmem-4', u'4 vCPUs, 26 GB RAM'),
 (u'n1-highmem-8', u'8 vCPUs, 52 GB RAM'),
 (u'n1-standard-1', u'1 vCPU, 3.75 GB RAM'),
 (u'n1-standard-16', u'16 vCPUs, 60 GB RAM'),
 (u'n1-standard-2', u'2 vCPUs, 7.5 GB RAM'),
 (u'n1-standard-32', u'32 vCPUs, 120 GB RAM'),
 (u'n1-standard-4', u'4 vCPUs, 15 GB RAM'),
 (u'n1-standard-8', u'8 vCPUs, 30 GB RAM')]

利用したいマシンタイプ名を設定する。

In [9]:
machine_type = 'f1-micro'

## イメージの指定

イメージの一覧を確認する。`project`には、利用したいイメージの所属プロジェクトを指定する。

In [10]:
images = compute.images().list(project='ubuntu-os-cloud').execute()['items']
images = filter(lambda i: i['status'] == 'READY' and 'deprecated' not in i, images)
map(lambda i: (i['name'], i['selfLink']), images)

[(u'ubuntu-1204-precise-v20160610a',
  u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1204-precise-v20160610a'),
 (u'ubuntu-1404-trusty-v20160610',
  u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20160610'),
 (u'ubuntu-1510-wily-v20160610',
  u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1510-wily-v20160610'),
 (u'ubuntu-1604-xenial-v20160610',
  u'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1604-xenial-v20160610')]

利用したいイメージのURLを設定する。

In [11]:
source_disk_image = 'https://www.googleapis.com/compute/v1/projects/ubuntu-os-cloud/global/images/ubuntu-1404-trusty-v20160610'

## キーペアの設定

現在のSSHキーの一覧を取得する。

In [12]:
projectMetadata = compute.projects().get(project=target_project_id).execute()
currentSSHKeys = filter(lambda metadata: metadata['key'] == 'sshKeys', projectMetadata['commonInstanceMetadata']['items']) \
                        if 'commonInstanceMetadata' in projectMetadata and 'items' in projectMetadata['commonInstanceMetadata'] else []
currentSSHKeys = currentSSHKeys[0]['value'].split('\n') if currentSSHKeys else []
currentSSHKeys

[u'ansible:ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX',
 u'']

SSHのキー一覧にこのNotebook環境のキーがなければ、追加する。

In [13]:
pub_key = None
with open(os.path.expanduser('~/.ssh/ansible_id_rsa.pub'), 'r') as f:
    pub_key = f.readlines()[0].strip()

if not filter(lambda k: k.endswith(pub_key), currentSSHKeys):
    currentSSHKeys.append('ansible:' + pub_key)
currentSSHKeys

[u'ansible:ssh-rsa XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX ansible@XXXXXXXXXXXX',
 u'']

Metadataに反映する。

In [14]:
compute.projects().setCommonInstanceMetadata(project=target_project_id,
                                             body={'items': [{'key': 'sshKeys',
                                                              'value': '\n'.join(currentSSHKeys)}]}).execute()

{u'id': u'8047257766995800862',
 u'insertTime': u'2016-06-17T07:49:21.362-07:00',
 u'kind': u'compute#operation',
 u'name': u'operation-1466174961053-5357a75bb6d49-9190a416-d7c15055',
 u'operationType': u'setMetadata',
 u'progress': 0,
 u'selfLink': u'https://www.googleapis.com/compute/v1/projects/my_project_id/global/operations/operation-1466174961053-5357a75bb6d49-9190a416-d7c15055',
 u'status': u'PENDING',
 u'targetId': u'201294258281',
 u'targetLink': u'https://www.googleapis.com/compute/v1/projects/my_project_id',
 u'user': u'me@developer.gserviceaccount.com'}

# マシンの確保

## インスタンスの起動

設定した情報を用いてマシンを確保する。

In [15]:
config = {
        'name': machine_name,
        'machineType': "zones/{}/machineTypes/{}".format(target_zone, machine_type),
        'disks': [
            {
                'boot': True,
                'autoDelete': True,
                'initializeParams': {
                    'sourceImage': source_disk_image,
                }
            }
        ],
        'networkInterfaces': [{
            'network': 'global/networks/default',
            'accessConfigs': [
                {'type': 'ONE_TO_ONE_NAT', 'name': 'External NAT'}
            ]
        }],
        'serviceAccounts': [{
            'email': 'default',
            'scopes': [
                'https://www.googleapis.com/auth/devstorage.read_write',
                'https://www.googleapis.com/auth/logging.write'
            ]
        }],
        'metadata': {
            'items': []
        }
    }

new_vm = compute.instances().insert(project=target_project_id, zone=target_zone, body=config).execute()
new_vm

{u'id': u'6984323738569912558',
 u'insertTime': u'2016-06-17T07:49:37.670-07:00',
 u'kind': u'compute#operation',
 u'name': u'operation-1466174976616-5357a76a8e641-b2fbf6a0-940630b9',
 u'operationType': u'insert',
 u'progress': 0,
 u'selfLink': u'https://www.googleapis.com/compute/v1/projects/my_project_id/zones/us-central1-f/operations/operation-1466174976616-5357a76a8e641-b2fbf6a0-940630b9',
 u'status': u'PENDING',
 u'targetId': u'7960517509428916462',
 u'targetLink': u'https://www.googleapis.com/compute/v1/projects/my_project_id/zones/us-central1-f/instances/test-gce',
 u'user': u'me@developer.gserviceaccount.com',
 u'zone': u'https://www.googleapis.com/compute/v1/projects/my_project_id/zones/us-central1-f'}

インスタンスがRunningになるまで待つ。

In [17]:
import time
status = compute.instances().get(project=target_project_id, zone=target_zone,
                                 instance=machine_name).execute()['status']
while status == 'PROVISIONING' or status == 'STAGING':
    time.sleep(30)
    status = compute.instances().get(project=target_project_id, zone=target_zone,
                                     instance=machine_name).execute()['status']
assert(status == 'RUNNING')

## IPアドレスの取得

In [18]:
vm_desc = compute.instances().get(project=target_project_id, zone=target_zone, instance=machine_name).execute()
ip_address = vm_desc['networkInterfaces'][0]['accessConfigs'][0]['natIP']
ip_address

u'XXX.XXX.XXX.59'

pingが通ることを確認。

In [21]:
!ping -c 4 {ip_address}

PING XXX.XXX.XXX.59 (XXX.XXX.XXX.59) 56(84) bytes of data.
64 bytes from XXX.XXX.XXX.59: icmp_seq=1 ttl=43 time=133 ms
64 bytes from XXX.XXX.XXX.59: icmp_seq=2 ttl=43 time=132 ms
64 bytes from XXX.XXX.XXX.59: icmp_seq=3 ttl=43 time=132 ms
64 bytes from XXX.XXX.XXX.59: icmp_seq=4 ttl=43 time=132 ms

--- XXX.XXX.XXX.59 ping statistics ---
4 packets transmitted, 4 received, 0% packet loss, time 3002ms
rtt min/avg/max/mdev = 132.871/133.058/133.458/0.233 ms


## Ansibleから操作可能であることを確認

[キーペアの設定](#キーペアの設定-3.6)でansibleユーザとしてこの環境の公開鍵をInjectionしている。

In [22]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

'/tmp/tmp9BOSSw'

In [23]:
import os
snapshot_hosts = os.path.join(work_dir, 'hosts')
with open(snapshot_hosts, 'w') as f:
    f.write('{address}\n'.format(address=ip_address)) 
!cat { snapshot_hosts }

XXX.XXX.XXX.59


Ansible経由でansibleユーザとしてSSHできることを確認する。

> インスタンス起動直後の場合、UNREACHABLEとなるおそれがある。その場合は再度実行すること。

In [24]:
!ansible -a 'whoami' -i { snapshot_hosts } all

XXX.XXX.XXX.59 | SUCCESS | rc=0 >>
ansible



# Inventoryの更新

Inventoryに、作成したマシンのIPアドレスを追加する。変更する前に、現在の内容をコピーしておく。

In [25]:
!cp inventory {work_dir}/inventory-old

[Inventory](../edit/inventory) を修正する。

In [26]:
!diff -ur {work_dir}/inventory-old inventory

--- /tmp/tmp9BOSSw/inventory-old	2016-06-17 23:52:39.873366095 +0900
+++ inventory	2016-06-17 23:52:56.545503890 +0900
@@ -3,3 +3,6 @@
 
 [test-vm]
 XXX.XXX.XXX.66
+
+[test-gce]
+XXX.XXX.XXX.59


追加したグループ名で、ansibleのpingモジュールが実行可能かどうかを確認する。

In [27]:
target_group = 'test-gce'

In [28]:
!ansible -m ping {target_group}

XXX.XXX.XXX.59 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


特にエラーとならなければOK。これで完了とする。

# 後始末

一時ディレクトリを削除する。

In [29]:
!rm -fr {work_dir}